## Imports

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pycountry
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from func_library import StackOverflowDataTester, StackOverflowData, AISalariesData, read_ppp, get_2023_usd_equivalent
from typing import TypeVar, Union
import pingouin as pg




# warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
np.random.seed(42)

Number = TypeVar('Number', int, float, np.float16, np.float32, np.float64, np.float128, np.int16, np.int32, np.int64, np.int8, np.uint16, np.uint32, np.uint64, np.uint8)

/var/folders/gw/9t1pgdqs0p172hmjwq0rgc2c0000gn/T/ipykernel_17013/3643884518.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Reading (Run Once)

In [3]:
ppp = read_ppp()
ai_salaries_df = AISalariesData.generate_df()
stack_overflow, skills_list, employments = StackOverflowData.generate_aggregate_df()


In [4]:
ai_jobs_converter = lambda row: get_2023_usd_equivalent(year=row["work_year"],
                                                        country_code=row["company_location"],
                                                        salary_val=row["salary"],
                                                        ppp_df=ppp)
ai_salaries_df["usd_2023_"] = ai_salaries_df.apply(ai_jobs_converter, axis=1)



Country code US or year 2023 not found in the PPP DataFrame


KeyError: 2023

,country,year,age
292,None,2020,35-44 years old
292,None,2020,35-44 years old
292,None,2020,35-44 years old
410,None,2020,18-24 years old
410,None,2020,18-24 years old
410,None,2020,18-24 years old
550,None,2020,25-34 years old
550,None,2020,25-34 years old
550,None,2020,25-34 years old
802,None,2020,nan


## Analysis

In [ ]:
# more workable until we go by skills
s_o = stack_overflow.drop(skills_list, axis=1)
list(s_o.columns)

['yearscode',
 'edlevel',
 'comptotal',
 'country',
 'employment',
 'convertedcompyearly',
 'mainbranch',
 'sopartfreq',
 'sovisitfreq',
 'age',
 'soaccount',
 'year',
 'yearscodepro',
 'count',
 'socomm',
 'surveyease',
 'orgsize',
 'surveylength',
 'analyst',
 'data scientist',
 'developer',
 'engineer_other',
 'management',
 'scientist_other',
 'systems_architect']

In [ ]:
ai_salaries_df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

### Descriptive Statistics

### One Way ANOVA on Location and Comp Salary

#### Helper Functions

In [ ]:
def get_similar_countries(df: pd.DataFrame, want: int = 10) -> list:
    """
    Get the number of countries that we want so we can index for ANOVA
    Consistent across the years for a good comparison
    """
    groupped = df.groupby('year')
    loc_list = list()
    head = 1
    
    while len(loc_list) < want:
        loc_set = set()
        for year, frame in groupped:
            grouped = frame.groupby("country").size().sort_values(ascending=False)
            if not loc_set:
                loc_set = set(grouped.head(head).index)
                continue
            temp_set = set(grouped.head(head).index)
            loc_set = loc_set.intersection(temp_set)
        loc_list = list(loc_set)
        head += 1
    return loc_list

#### ANOVA

In [ ]:
loc_so = s_o[["year", "country", "convertedcompyearly"]]
loc_so.loc[:, "year"] = loc_so.loc[:, "year"].astype('int64')

loc_sal = ai_salaries_df[["work_year", "employee_residence", "salary_in_usd"]]
loc_sal.loc[:, "salary_in_usd"] = loc_sal.loc[:, "salary_in_usd"].astype("float64")
loc_sal.columns = loc_so.columns


location = pd.concat([loc_so, loc_sal], ignore_index=True)
location
loc_so.dtypes, loc_sal.dtypes

(year                    object
 country                 object
 convertedcompyearly    float64
 dtype: object,
 year                    int64
 country                object
 convertedcompyearly     int64
 dtype: object)

In [ ]:
countries = get_similar_countries(location, 20)

In [ ]:
for year, frame in groupped:
    print(year)
    frame = frame[frame["country"].isin(countries)]
    aov = pg.anova(data=frame, dv="convertedcompyearly", between="country", detailed=True)
    display(aov)

2019


,Source,SS,DF,MS,F,p-unc,np2
0,country,1.044744e+14,19,5.498651e+12,51.971066,6.123095e-190,0.069654
1,Within,1.395425e+15,13189,1.058022e+11,NaN,NaN,NaN


2020


,Source,SS,DF,MS,F,p-unc,np2
0,country,4.377320e+13,19,2.303853e+12,34.972102,1.331925e-123,0.071909
1,Within,5.649601e+14,8576,6.587688e+10,NaN,NaN,NaN


2021


,Source,SS,DF,MS,F,p-unc,np2
0,country,4.776961e+13,19,2.514190e+12,14.355395,3.937604e-46,0.027638
1,Within,1.680634e+15,9596,1.751390e+11,NaN,NaN,NaN


2022


,Source,SS,DF,MS,F,p-unc,np2
0,country,8.976899e+13,19,4.724684e+12,6.682907,7.586757e-18,0.013915
1,Within,6.361409e+15,8998,7.069803e+11,NaN,NaN,NaN


2023


,Source,SS,DF,MS,F,p-unc,np2
0,country,1.271251e+13,19,6.690794e+11,55.024831,4.765940e-200,0.085467
1,Within,1.360293e+14,11187,1.215959e+10,NaN,NaN,NaN


### T-Tests On Salary if Needed

### One Way ANOVA on Degree

### One Way ANOVA on Experience

### One Way ANOVA on Location

### One Way ANOVA on Year

### Salary Skew Overall

### Identify any outliers

## Charts

### Choropleth Chart on Salary by Location (need)

### Line Chart of Salary by Location (need)

### Blox Plot of a Country to Show Skew if Any (need)

### Histogram of Countries with Highest Response Rates (need)

### Sunburst Plot (want)